# tf_idf_similarity MulitiClassifier Model

1. 데이터  
    * data_0112를 사용함  
    * data_0105의 text_response를 추가함  
# 
2. tf-idf similarity matrix 
    * STEP 1 : train set과 test set을 합쳐서 tf_idf_matrix를 구하고 서로의 simliarity matrix를 구함
    * STEP 2 : test_set의 특정 레코드와 가장 비슷한 레코드를 train_set에서 찾고 그 레코드의 knowcode를 결과값으로 내놓기
#          
3. fitting & prediction
    * STEP 1 : 각 연도의 전체 데이터를 바탕으로 et_100을 학습시킨다
    * STEP 2 : threshold를 기준으로, 유사도가 그 보다 높은 레코드에 대해서만 similarity matrix로 예측을 진행함
    * STEP 3 : threshold를 기준으로, 유사도가 그 보다 낮은 레코드에 대해서, et_100으로 예측을 진행함

# 라이브러리 import & 데이터 로딩 & 전처리

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from glob import glob
import warnings
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier

warnings.filterwarnings(action='ignore') 
pd.options.display.max_columns=None

know_train = [pd.read_csv(path) for path in sorted(glob('../data_0112/train/*.csv'))]
know_test = [pd.read_csv(path) for path in sorted(glob('../data_0112/test/*.csv'))]

other_train = [pd.read_csv(path) for path in sorted(glob('../data_0105/train/*.csv'))]
other_test = [pd.read_csv(path) for path in sorted(glob('../data_0105/test/*.csv'))]

for num in range(4):
    know_train[num]['text_response'] = other_train[num]['text_response']
    know_train[num]['major'] = other_train[num]['major']
    know_test[num]['text_response'] = other_test[num]['text_response']
    know_test[num]['major'] = other_test[num]['major']

# tf-idf similarity matrix

In [41]:
# STEP 1 : 같은 연도인 train_set, test_set으로 sim_df 만들기
# input
def get_tf_idf_sim_mat(train_data, test_data):
    '''
    TRAIN SET과 TEST SET을 합치고 그에 대한 tf_idf_matrix를 도출한 후, similarity matrix를 구함
    '''
    total_nouns_list = []

    doc_nouns_list_train = [doc for doc in (train_data['text_response'] +' '+ train_data['major'])]
    doc_nouns_list_test = [doc for doc in (test_data['text_response'] +' '+ test_data['major'])]

    total_nouns_list.extend(doc_nouns_list_train)
    total_nouns_list.extend(doc_nouns_list_test)

    stopwords = ['없다', '공란', '0']
    tfidf_vectorizer = TfidfVectorizer(min_df=1,stop_words=stopwords)
    tfidf_matrix = tfidf_vectorizer.fit_transform(total_nouns_list)
    doc_nouns_similarities = (tfidf_matrix * tfidf_matrix.T)
    sim_df = pd.DataFrame(doc_nouns_similarities.toarray())
    
    return sim_df

# STEP 2 : test_set의 특정 레코드와 가장 비슷한 레코드를 train_set에서 찾고 그 레코드의 knowcode를 결과값으로 내놓기
def sim_pred(idx, sim_df, threshold, train_data):
    '''
    test_set의 특정 레코드와 가장 비슷한 레코드를 train_set에서 찾고 그 레코드의 knowcode를 결과값으로 내놓기
    '''
    
    train_len = train_data.shape[0]
    
    test_set_record = idx + train_len   # sim_df에서 test_set의 index를 계산
    sim_rank_series = sim_df.loc[test_set_record].sort_values(ascending=False) 
    filtered_sim_rank_series = sim_rank_series.loc[sim_rank_series.index < train_len]  # test_set_record와 가장 비슷한 train_set_record를 계산

    target_index = list(filtered_sim_rank_series.head(1).index)[0]
    target_similarity = list(filtered_sim_rank_series.head(1).values)[0]

    if target_similarity > threshold:   # 조건을 만족한다면 유사한 train_set의 knowcode와 같은 것으로 예측
        pred = train_data.loc[target_index,'knowcode']
        
    else:   # 조건을 만족하지 않는다면 건너뛰었다는 의미로 0을 반환
        pred = 0
    
    return pred
    

# fitting & prediction

In [30]:
# STEP 1 : 각 연도의 전체 데이터를 바탕으로 et_100을 학습시킨다
years = ['2017','2018','2019','2020']
train_data = {}

for year, df in zip(years, know_train):
    train_data[year] = {'X': df.drop(['text_response','idx','major','knowcode'], axis=1),
                        'y': df['knowcode']}
    
RANDOM_STATE = 42
et_models = {}

for year in tqdm(years):
    model = ExtraTreesClassifier(n_estimators=100, random_state=RANDOM_STATE, n_jobs=8)
    model.fit(train_data[year]['X'], train_data[year]['y'])
    et_models[year] = model

# test data 준비하기
test_data = {}

for year, df in zip(years, know_test):
    train_columns = train_data[year]['X'].columns
    test_data[year] =  {'X': df[train_columns]}

100%|██████████| 4/4 [00:51<00:00, 12.77s/it]


In [105]:
# STEP2 : threshold를 기준으로, 유사도가 그 보다 높은 레코드에 대해서만 similarity matrix로 예측을 진행함

sim_predicts = [] 
threshold = 0.5

for year, num in tqdm(zip(years,range(4))):
    sim_df = get_tf_idf_sim_mat(know_train[num], know_test[num])    # sim_df를 먼저 구해주고
    sim_predict = []
    
    for idx in range(test_data[year]['X'].shape[0]):
        pred = sim_pred(idx, sim_df, threshold, know_train[num])
        sim_predict.append(pred)

    sim_predicts.extend(sim_predict)
    
submission = pd.read_csv('../data_0103/sample_submission.csv') # sample submission 불러오기
submission['knowcode'] = sim_predicts

4it [01:35, 23.87s/it]


In [107]:
# STEP3 : threshold를 기준으로, 유사도가 그 보다 낮은 레코드에 대해서, et_100으로 예측을 진행함

non_sim_predicts = []
idx_start = 0
idx_end = 0

final_submission = pd.DataFrame()
for year in tqdm(years):
    test_len = test_data[year]['X'].shape[0] - 1
    idx_end = idx_start + test_len
    submission_year = submission.loc[idx_start:idx_end,:].reset_index(drop=True)
    idx_start = idx_start + test_len + 1
    
    zero_indice = submission_year[submission_year['knowcode']==0].index     # 이전 예측에서 0으로 지정했던 데이터의 인덱스
    
    test_data_zero = test_data[year]['X'].loc[zero_indice,:]    # 그것들만 따로 추려서 model에 넣어 예측해주고
    pred = et_models[year].predict(test_data_zero)
    
    submission_year.loc[zero_indice,'knowcode'] = pred      # submission에 채워줍니다
    final_submission = pd.concat([final_submission, submission_year])
    
final_submission = final_submission.reset_index(drop=True)

100%|██████████| 4/4 [00:11<00:00,  2.98s/it]


In [108]:
final_submission.to_csv('et_data0112_tf_idf_matrix_sim_ver3.csv', index=False)